## Peakdeck: Choosing the Best Parameters
As part of the HIVDC collaboration with Alex Rives, we are interepreting ATACSeq data from the Littman Lab at NYU Med. In order to transform ATACSeq High Throughput Sequencing data into Transcription Factor Activity priors, we need to infer chromatin accessibility from peaks of sequencing reads. We do this by first aligning the Fastq data with Bowtie, then calling peaks with Kernel-Density-Estimator-based <a> Peakdeck </a href=http://www.ncbi.nlm.nih.gov/pubmed/24407222> 

### Pipeline overview
Fastq -> Sam -> Bam -> filtered Bam (removing Mitochondrial DNA) -> Sam -> Peakdeck output -> Bed -> Wig.

We also generate a tdf from the filtered bam file

We've pulled DNAseI hypersensitivity sites from Encode using the following protocol: Downloaded from http://hgdownload-test.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgDnaseMasterSites/releaseLatest/wgEncodeAwgDnaseMasterSites.bed.gz, then lifted over to hg38 using the chain file from http://hgdownload.cse.ucsc.edu/gbdb/hg19/liftOver/hg19ToHg38.over.chain.gz

The bed master file was sorted and filtered to only have chromosomes whose names exist in the HG38 assembly and sorted

Running this on our cluster gives the following genome-wide output:

| depth | length | total | fraction  | 
|------|--------|-------|----|
| 0 | 2663458615 | 3099750718 | 0.859 |
| 1 | 436088249 | 3099750718 | 0.141 |
| 2 | 203511 | 3099750718 | 6.57e-05 |
| 3 | 343 | 3099750718 | 1.11e-07 |

This shows that the Encode hypersensitivy regions cover slightly under 15% of the genome, with a very small number of self-overlaps. 

### Sanity Check
Let's run this on the orignal (pre-lift over) hg19 bed file and verify that this depth signature is approximately the same. 

Genome-wide Output:

| depth | length | total | fraction  | 
|------|--------|-------|----|
| 0 | 2703081244 | 3137161264 | 0.862 |
| 1 | 434080020 | 3137161264 | 0.138 |

Therefore this ~200kb overlap of hypersensitivity sites is the result of conversion from hg19 to hg38

The quick fix I'll use for now is to use bedtools merge with a -d 10 (this will merge intervals within 10 bps). http://bedtools.readthedocs.io/en/latest/content/tools/merge.html

| depth | length | total | fraction  | 
|------|--------|-------|----|
| 0 | 2661936787 | 3099750718 | 0.859 |
| 1 | 437813931 | 3099750718 | 0.141 |

### Problem Setup
We will set the Encode hypersensitivy regions as the gold standard. Predicted peaks from peakdeck that overlap with these regions will be considered TPs, predicted peaks that don't overlap will be considered FPs, and hypersensitivy regions that don't overlap with any predicted peaks will be FNs. 

### TBD
We have data from 22 samples. Should their peaks be combined? Let's first do a quick sanity check to make sure there aren't outliers. We can look at the peakdeck output bed files for fraction of intersections out of all intervals, and do a heatmap of distance across all samples. 

### Sample peakdeck output

Tricky bit: I want to merge this bed file, but I want to keep the lowest p-value for merged regions. Let's see what happens if I do a naive merge: 

### Relative size of each file
Our merged, sorted and filtered hypersensitivity regions file is now 2713878 lines long, and the peak deck output ranges from 20k - 90k lines.

In [11]:
# Fraction of peaks called by peakdeck in hypersensitivity sites:
13457293 / float(31169325)

0.43174797657632946

In [12]:
SAMPLE200BED=out/out/24h_mock_D185_20150601_S10/24h_mock_D185_20150601_S10.peakdeck.200bps.10000b.out.bed
$BEDTOOLS merge -i $SAMPLE200BED > new_sample_200.bed
$BEDTOOLS intersect -f 1E-3 -wb -a merged_sorted_filtered_wgEncodeAwgDnaseMasterSites.hg38.bed -b new_sample_200.bed > intersect.bed
24667275

cat intersect.bed | awk -F'\t' 'BEGIN{SUM=0}{ SUM+=$3-$2 }END{print SUM}'
11463669

SyntaxError: invalid syntax (<ipython-input-12-c2e413db6688>, line 1)

In [13]:
# Fraction of peaks called by peakdeck in hypersensitivity sites:
11463669 / float(24667275)

0.4647318765449366

In [14]:
$BEDTOOLS intersect -f 1E-3 -wb -a merged_sorted_filtered_wgEncodeAwgDnaseMasterSites.hg38.bed -b new_sample.bed > intersect.bed
cat intersect.bed | awk -F'\t' 'BEGIN{SUM=0}{ SUM+=$3-$2 }END{print SUM}'
10162500
genome	0	3078847238	3099750718	0.993256
genome	1	20903480	3099750718	0.0067436

SyntaxError: invalid syntax (<ipython-input-14-488ca53956af>, line 1)

In [15]:
# Fraction of peaks called by peakdeck in hypersensitivity sites:
10162500 / float(20903480)

0.4861630694984759

In [ ]:
SAMPLEBED=out/out/24h_mock_D185_20150601_S10/24h_mock_D185_20150601_S10.peakdeck.160bps.10000b.out.bed
$BEDTOOLS merge -i $SAMPLEBED > new_sample.bed
$BEDTOOLS intersect -f 1E-3 -wb -a merged_sorted_filtered_wgEncodeAwgDnaseMasterSites.hg38.bed -b new_sample.bed > intersect.bed
cat intersect.bed | awk -F'\t' 'BEGIN{SUM=0}{ SUM+=$3-$2 }END{print SUM}'
10960265
cat new_sample.bed | awk -F'\t' 'BEGIN{SUM=0}{ SUM+=$3-$2 }END{print SUM}'
22601945

In [10]:
# Fraction of peaks called by peakdeck in hypersensitivity sites:
10960265 / float(22601945)

0.4849257442224552

In [18]:
6430249/ float(11918125)

0.5395352876396246